In [4]:

import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchbearer
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchbearer import Trial

In [5]:
torch.cuda.is_available()

True

In [6]:
# convert each image to tensor format
transform = transforms.Compose([
    transforms.ToTensor()  # convert to tensor
])

# load data
trainset = MNIST(".", train=True, download=True, transform=transform)
testset = MNIST(".", train=False, download=True, transform=transform)

# create data loaders
trainloader = DataLoader(trainset, batch_size=128, shuffle=True)
testloader = DataLoader(testset, batch_size=128, shuffle=True)

100%|██████████| 9.91M/9.91M [00:01<00:00, 9.44MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 272kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 2.57MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 29.1MB/s]


In [7]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, (5, 5), padding=0)
        self.fc1 = nn.Linear(32 * 12**2, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        out = self.conv1(x)
        out = F.relu(out)
        out = F.max_pool2d(out, (2,2))
        out = F.dropout(out, 0.2)
        out = out.view(out.shape[0], -1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        return out

In [8]:
# build the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleCNN()
model = model.to(device)
# define the loss function and the optimiser
loss_function = nn.CrossEntropyLoss()
optimiser = optim.Adam(model.parameters())

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(trainloader, test_generator=testloader)
trial.run(epochs=5)
results = trial.evaluate(data_key=torchbearer.TEST_DATA)
print(results)

0/5(t):   0%|          | 0/469 [00:00<?, ?it/s]

1/5(t):   0%|          | 0/469 [00:00<?, ?it/s]

2/5(t):   0%|          | 0/469 [00:00<?, ?it/s]

3/5(t):   0%|          | 0/469 [00:00<?, ?it/s]

4/5(t):   0%|          | 0/469 [00:00<?, ?it/s]

0/1(e):   0%|          | 0/79 [00:00<?, ?it/s]

{'test_loss': 0.04051032289862633, 'test_acc': 0.986799955368042}
